In [19]:
# Importando as bibliotecas

In [20]:
import requests as r
import pandas as pd
from datetime import datetime
import numpy as np
from pandasql import sqldf
import pyodbc as odc


# Requisitando os dados na API

In [21]:
url = 'https://covid19-brazil-api.now.sh/api/report/v1'
df = r.request('GET',url)

# Convertendo o Json em um DataFrame

In [27]:
#Criando um DataFrame vazio.
df2 = pd.DataFrame()
i = 0
    
#Obetendo os dados da API
df = r.request('GET',url)
    
#Obtendo a data atual do sistema.
data_atual = datetime.today().strftime('%Y-%m-%d')

#Lendo linha a linha do dicionaro 'Data' vindo do arquivo Json.
#O dicionario e composto por chave e valor.

for i in range(len(df.json()['data'])):
        df1 = pd.DataFrame(df.json()['data'][i], index=[0])

        #Alterando o formato da data
        df1['datetime'] = pd.to_datetime(df1['datetime']).dt.strftime("%Y-%m-%d")

        #Adicionando os registros ao dataframe = df2
        df2 = df2.append(df1)


str

# Fazer Select no Banco para conferir valores repetidos.

In [28]:
def SelectBanco ():
    
    con_string = ('DRIVER={SQL Server};'
              'SERVER=DESKTOP-AF7BA3M;'
              'DATABASE=CovidDB;')

    cnxn = odc.connect(con_string)
    query = """SELECT * FROM Tb_Covid1"""
    #Passando o retorno da consulta para um dataframe = 'result'
    result = pd.read_sql(query, cnxn)
    return result

# Função para verificar a ultima data dos registros.

In [36]:
def IncrementoBanco (query,data_atual,df2):
    df0 = pd.DataFrame(query)
    if (df2['datetime'].max() != df0['datetime'].max()) :
            GravarNoBanco(df2)   

# Insert - SQL

In [30]:
def GravarNoBanco(df):

        cnxn = odc.connect('DRIVER={SQL Server};'
                       'SERVER=DESKTOP-AF7BA3M;'
                        'DATABASE=CovidDB;')
        cursor = cnxn.cursor()

        for index , linha in df.iterrows():
            cursor.execute("insert into Tb_Covid1(uid, uf, state,cases,deaths,suspects,refuses,datetime) values (?,?,?,?,?,?,?,?)",linha.uid,linha.uf,linha.state,linha.cases,linha.deaths,linha.suspects,linha.refuses,linha.datetime)
            cursor.commit()

# Instancioando as funçoes.

In [37]:
# Query no banco de dados verificando a ultima data.
query = SelectBanco()

#Incrementando os dados no banco
IncrementoBanco(query,data_atual,df2)

# Consultando o banco para verificar os valores inseridos.

In [41]:
query = SelectBanco()
query['datetime'].max()
query.head(50)

,id,uid,uf,state,cases,deaths,suspects,refuses,datetime
0,1001,35,SP,São Paulo,5362377,167989,5334,596,2022-04-22
1,1002,31,MG,Minas Gerais,3352544,61216,925,104,2022-04-22
2,1003,41,PR,Paraná,2441061,43072,400,119,2022-04-22
3,1004,43,RS,Rio Grande do Sul,2319676,39250,416,330,2022-04-22
4,1005,33,RJ,Rio de Janeiro,2131863,73207,1254,148,2022-04-22
5,1006,42,SC,Santa Catarina,1696325,21738,346,47,2022-04-22
6,1007,29,BA,Bahia,1540547,29831,573,36,2022-04-22
7,1008,52,GO,Goiás,1320210,26411,353,55,2022-04-22
8,1009,23,CE,Ceará,1243770,26867,493,89,2022-04-22
9,1010,32,ES,Espírito Santo,1045921,14391,71,21,2022-04-22
